# Load Libraries

In [5]:
import os
import warnings
import logging
import sys
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO)

# Suppress warnings
warnings.filterwarnings("ignore")

# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None)

# Load Data

In [ ]:
# Load Data From Pickle
with open('../../Data/Data_Iran_1983_2022.pkl', 'rb') as f:
    data = pickle.load(f)
    logging.info("Data loaded from pickle file.")

INFO:root:Data loaded from pickle file.


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2454681 entries, 0 to 2454680
Data columns (total 17 columns):
 #   Column             Dtype         
---  ------             -----         
 0   station_id         category      
 1   station_name       category      
 2   region_id          category      
 3   region_name        category      
 4   lat                float64       
 5   lon                float64       
 6   station_elevation  float64       
 7   date               datetime64[ns]
 8   tmax               float64       
 9   tmin               float64       
 10  tm                 float64       
 11  umax               float64       
 12  umin               float64       
 13  um                 float64       
 14  ffm                float64       
 15  sshn               float64       
 16  rrr24              float64       
dtypes: category(4), datetime64[ns](1), float64(12)
memory usage: 257.5 MB


# Calculate ETo

In [11]:
import pyet

def calculate_eto(group):
    print(f"Calculating ETo for station {group.name}")
    group = group.set_index('date').sort_index()
    tmean = group['tm']
    wind = group['ffm']
    lat = pyet.utils.deg_to_rad(lat=group['lat'].iloc[0])
    elevation = group['station_elevation'].iloc[0]
    rs = pyet.calc_rad_sol_in(n=group['sshn'], lat=lat, as1=0.25, bs1=0.5, nn=None)
    tmax = group["tmax"]
    tmin = group["tmin"]
    rhmax = group["umin"]
    rhmin = group["umax"]
    rh = group["um"]
    # try:
    eto = pyet.calculate_all(
        tmean=tmean,
        wind=wind,
        rs=rs,
        elevation=elevation,
        lat=lat,
        tmax=tmax,
        tmin=tmin,
        rh=rh,
        rhmax=rhmax,
        rhmin=rhmin
    )
    return eto
    # except Exception as e:
    #     print(f"Error with simple method for station {group.name}: {e}")
    #     return pd.Series(np.nan, index=group.index)

data['eto_simple'] = data.groupby(
    ['region_id', 'station_id'],
    group_keys=False
).apply(calculate_eto)


Calculating ETo for station ('ALKK', '40752')
Calculating ETo for station ('ALKK', '88125')
Calculating ETo for station ('ALKK', '99321')
Calculating ETo for station ('ALKK', '99373')
Calculating ETo for station ('ALKK', '99396')
Calculating ETo for station ('MAAA', '40769')
Calculating ETo for station ('MAAA', '40891')


Exception: The radiation input provided is greater than 100 MJ/m2d, which is not realistic. Please convert the radiation input to MJ/m2d.

In [12]:
data

,station_id,station_name,region_id,region_name,lat,lon,station_elevation,date,tmax,tmin,tm,umax,umin,um,ffm,sshn,rrr24
0,40757,Semnan,OIIS,Semnan,35.59,53.42,1127.00,1983-01-01 12:00:00,8.40,-0.80,4.60,73.00,41.00,58.00,0.00,7.00,0.10
1,40757,Semnan,OIIS,Semnan,35.59,53.42,1127.00,1983-01-03 12:00:00,8.80,0.40,4.60,89.00,45.00,63.33,0.00,7.70,0.00
2,40757,Semnan,OIIS,Semnan,35.59,53.42,1127.00,1983-01-04 12:00:00,5.40,-1.00,1.70,67.00,51.00,60.00,0.00,8.20,0.00
3,40757,Semnan,OIIS,Semnan,35.59,53.42,1127.00,1983-01-05 12:00:00,4.80,-4.00,0.40,69.00,47.00,52.67,0.00,8.30,0.00
4,40757,Semnan,OIIS,Semnan,35.59,53.42,1127.00,1983-01-06 12:00:00,5.60,-3.20,1.80,68.00,39.00,49.17,0.50,8.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2454676,40840,Marvast,OIYY,Yazd,30.46,54.21,1547.00,2022-12-17 12:00:00,14.50,0.10,8.50,80.00,40.00,61.00,3.80,6.40,0.00
2454677,40840,Marvast,OIYY,Yazd,30.46,54.21,1547.00,2022-12-19 12:00:00,11.90,-1.50,6.70,91.00,38.00,59.40,2.40,8.60,0.00
2454678,40840,Marvast,OIYY,Yazd,30.46,54.21,1547.00,2022-12-20 12:00:00,14.10,-2.30,7.00,81.00,24.00,48.00,1.40,9.40,0.00
2454679,40840,Marvast,OIYY,Yazd,30.46,54.21,1547.00,2022-12-21 12:00:00,12.70,-4.40,6.20,72.00,21.00,41.40,2.00,9.20,0.00


# Export Data

In [ ]:
# # To pickle in data folder
# with open('../../Data/Data_Iran_1983_2022.pkl', 'wb') as f:
#     pickle.dump(df, f)